## Load + process data

In [ ]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.cd("./SSC_ADMM_v1.1")

In [ ]:
# I renamed files 1, 2.. to 01, 02..
# so that they are globbed in order later

from pathlib import Path
def rename_Coil20(path="./data/coil-20-proc"):
    path = Path(path)
    for img in path.glob("obj*.png"):
        name = img.name
        new_name = name
        
        if(name.find('__') < 5):
            new_name = new_name[:3] + '0' + new_name[3:]
        if(len(new_name) < 13):
            new_name = new_name[:7] + '0' + new_name[7:]
        if(new_name != name):
            print("Renamed " + name + " to " + new_name)
            img.rename(path / new_name)

rename_Coil20()

In [ ]:
from load import load_Coil20

images_raw, labels = load_Coil20()

In [ ]:
data_h = 32
data_w = 32

In [ ]:
import numpy as np
from skimage.transform import resize

images_compressed = np.moveaxis(resize(np.moveaxis(images_raw, 0, -1), output_shape=(32, 32),
                                       order=1, mode='reflect', anti_aliasing=True), -1, 0)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2)

ax[0].imshow(images_raw[0]);
ax[1].imshow(images_compressed[0]);

In [ ]:
fig, ax = plt.subplots(1, 2)
idx = 0

ax[0].imshow(np.mean(images_raw[72*idx:72*(idx+1)], axis=0));
ax[1].imshow(np.mean(images_compressed[72*idx:72*(idx+1)], axis=0));

In [ ]:
fig, ax = plt.subplots(1, 2)

ax[0].imshow(np.mean(images_raw, axis=0));
ax[1].imshow(np.mean(images_compressed, axis=0));

In [ ]:
from visualize import tSNE_2D, tSNE_3D

In [ ]:
images_raw_flat = images_raw.reshape(images_raw.shape[0], -1)
images_compressed_flat = images_compressed.reshape(images_compressed.shape[0], -1)
tSNE_2D(images_compressed_flat, labels)
#tSNE_3D(images_compressed_flat, labels)

## Normalize data

In [ ]:
inpt = images_compressed_flat

# global rescale to [-1, 1]
mmin = np.min(inpt)
mmax = np.max(inpt)
images_norm = (np.multiply(inpt, 2, dtype='float64') - mmax - mmin) / (mmax - mmin)

In [ ]:
from load import split_mult

if('images_norm_val' not in locals()):
    validation, training = split_mult([images_norm, labels], 0.2)
    images_norm, labels = training
    images_norm_val, labels_val = validation

In [ ]:
# from scipy.io import savemat

# savemat('./saved/raw/Coil20.mat', mdict={'X':images_raw, 'Y':labels})
# savemat('./saved/rescaled/Coil20.mat', mdict={'X':images_norm, 'Y':labels, 'X_val':images_norm_val, 'Y_val':labels_val})

## Calculate C matrix

In [ ]:
import numpy as np
import supporting_files.sda as sda

from supporting_files.helpers import optimize
from scipy.io import savemat, loadmat

In [ ]:
# Matlab SSC #1
savemat('./temp.mat', mdict={'X': images_norm})
k = len(np.unique(labels))
alpha = 10.0
maxIter = 63
eng.SSC_modified(k, 0, False, alpha, False, 1, 1e-20, maxIter, False, 0)
C = loadmat("./temp.mat")['C']

In [ ]:
display_image(C[:110, :110], 110, 110)
print(np.mean(np.square(C)))

In [ ]:
fig, ax = plt.subplots(2)

index = 0;
ax[0].imshow(images_norm[index].reshape((data_h, data_w)));
ax[1].imshow(np.matmul(C, images_norm)[index].reshape((data_h, data_w)));
print(np.mean(np.square(images_norm - np.matmul(C, images_norm))))

## Train Autoencoder

In [ ]:
from supporting_files import helpers
from supporting_files import sda
import dsc
import importlib
importlib.reload(helpers)
importlib.reload(sda)
importlib.reload(dsc)

In [ ]:
%%time
trainC = False
C = None
d = dsc.DeepSubspaceClustering(images_norm, images_norm_val, C=C, trainC=trainC, hidden_dims=[32], weight_init='sda-normal',
                               weight_init_params={'epochs_max':10000, 'sda_printstep':100, 'validation_step':10}, lr=0.001, batch_num=1,
                               sda_optimizer='Adam', sda_decay='none', verbose=False, save_path="./saved/models/coil20/test_{0:.4g}")

In [ ]:
%%time
trainC = True
C = C
d = dsc.DeepSubspaceClustering(images_norm, images_norm_val, C=C, trainC=trainC, hidden_dims=[32],
                               lr=0.001, batch_num=1, seed=0, verbose=True, load_path="./saved/models/coil20/test_0.1699")

In [ ]:
encoded_h = 4
encoded_w = 8

In [ ]:
%%time
sess = d.train(lambda1=1.0, lambda2=1.0, lambda3=1.0, learning_rate=0.00001, optimizer='Adam', decay='none',
               batch_num=100, epochs=1000, print_step=100, validation_step=10, stop_criteria=3)
images_HM2 = d.result
images_HM = d.reconstr
if(trainC):
    trained_C = np.float64(d.outC)
else:
    trained_C = C

In [ ]:
import matplotlib.pyplot as plt

def display_image(image, height, width):
    print(np.min(image), np.max(image))
    imgplot = plt.imshow(image.reshape((height, width)))

In [ ]:
# lambda3 - regularization on trained_C
if(trained_C is not None):
    display_image(trained_C[:110, :110], 110, 110)

In [ ]:
display_image(images_HM2[0], encoded_h, encoded_w)

In [ ]:
# lambda1 - self-expressiveness
if(trained_C is not None):
    display_image(np.matmul(trained_C, images_HM2)[0], encoded_h, encoded_w)
    print(np.mean(np.square(images_HM2 - np.matmul(trained_C, images_HM2))))

In [ ]:
display_image(np.mean(images_HM2, axis=0), encoded_h, encoded_w)

In [ ]:
display_image(np.std(images_HM2, axis=0), encoded_h, encoded_w)

In [ ]:
tSNE_2D(images_HM2, labels)
# tSNE_3D(images_HM2, labels)

In [ ]:
# AE Reconstruction
fig, ax = plt.subplots(1, 2)

index = 0;
ax[0].imshow(images_norm[index].reshape((data_h, data_w)));
ax[1].imshow(images_HM[index].reshape((data_h, data_w)));

In [ ]:
fig, ax = plt.subplots(1, 2)

ax[0].imshow(np.std(images_norm, axis=0).reshape((data_h, data_w)));
ax[1].imshow(np.std(images_HM, axis=0).reshape((data_h, data_w)));
print("By rows (original):", np.mean(np.std(images_norm, axis=0).reshape((data_h, -1)), axis=1))
print("By rows (AE reconstr):", np.mean(np.std(images_HM, axis=0).reshape((data_h, -1)), axis=1))

In [ ]:
# Orthogonalize HM2 with PCA
from sklearn.decomposition import PCA
import numpy as np

pca2 = PCA(n_components=30, whiten=False, svd_solver='arpack', random_state=0)
images_HM2_orth = pca2.fit_transform(images_HM2)

images_HM2_orth.shape

In [ ]:
stds2 = np.std(images_HM2_orth, axis=0)
display_image(stds2, 5, 6)
print("By rows:", np.mean(stds2.reshape((6, -1)), axis=1))

In [ ]:
np.std(images_HM2_orth, axis=0)[1] / np.std(images_HM2_orth, axis=0)[0]

In [ ]:
# AE Features (orthogonalized)
from ipywidgets import BoundedIntText, FloatSlider, Output, VBox
idx = BoundedIntText(description="Index:", max=images_HM2_orth.shape[1]-1)
val = FloatSlider(description="Value:", continuous_update=False)
output = Output()

indx = 0
vector = images_HM2_orth[indx:indx+1].copy()
val_mins = (np.mean(images_HM2_orth, axis=0) - 2 * np.std(images_HM2_orth, axis=0)).flatten()
val_maxs = (np.mean(images_HM2_orth, axis=0) + 2 * np.std(images_HM2_orth, axis=0)).flatten()

def update_channels(change):
    val.min = val_mins[change.new]
    val.max = val_maxs[change.new]
    val.step = (val_maxs[change.new] - val_mins[change.new]) / 100
    val.value = vector[0][change.new]
idx.observe(update_channels, 'value')

def update_plot(change):
    if(change is not None):
        vector[0][idx.value] = change.new
    output.clear_output(wait=True)
    with output:
        plt.imshow(vector[0].reshape((5, 6)))
        plt.show()
        out = sess.run(d.H_M, feed_dict={d.H_M_2_post: pca2.inverse_transform(vector)})
        plt.imshow(out.reshape((data_h, data_w)))
        plt.show()
val.observe(update_plot, 'value')

update_channels(type('obj', (object,), {'new': 0}))
VBox([idx, val, output])

In [ ]:
fig, ax = plt.subplots(1, 2)

A = 0
B = 800

ax[0].imshow(images_norm[A].reshape(32, -1))
ax[1].imshow(images_norm[B].reshape(32, -1))

In [ ]:
fig, ax = plt.subplots(1, 2)

ax[0].imshow(sess.run(d.H_M, feed_dict={d.H_M_2_post: images_HM2[A:A+1]}).reshape(32, -1))
ax[1].imshow(sess.run(d.H_M, feed_dict={d.H_M_2_post: images_HM2[B:B+1]}).reshape(32, -1))

In [ ]:
from IPython import display

for i in range(51):
    f = i / 50
    intermed = (1-f)*images_HM2[A:A+1] + f*images_HM2[B:B+1]
    plt.imshow(sess.run(d.H_M, feed_dict={d.H_M_2_post: intermed}).reshape(32, -1))
    display.clear_output(wait=True)
    display.display(plt.gcf())

display.clear_output(wait=True)

In [ ]:
# HM2 Rotation
fig, ax = plt.subplots(6, 4)

for i in range(24):
    ax[i%6][i//6].imshow(images_HM2[2*i].reshape((encoded_h, encoded_w)));
    ax[i%6][i//6].set_title(2*i)

In [ ]:
# Matlab SSC #2
k = len(np.unique(labels))
alpha = 20.0
maxIter = 63
if(not trainC):
    savemat('./temp.mat', mdict={'X': images_HM2})
else:
    savemat('./temp.mat', mdict={'C': trained_C})
grps = eng.SSC_modified(k, 0, False, alpha, False, 1, 1e-20, maxIter, True, 0, trainC)
C_after = loadmat("./temp.mat")['C']
labels_pred = np.asarray(grps, dtype=np.int32).flatten()

In [ ]:
from ipywidgets import IntSlider, Output, VBox
sld_x = IntSlider(description="X:", max=C_after.shape[1]-110, continuous_update=True)
sld_y = IntSlider(description="Y:", max=C_after.shape[0]-110, continuous_update=True)
output = Output()

def update_C(change):
    output.clear_output(wait=True)
    with output:
        plt.imshow(C_after[sld_y.value:sld_y.value+110, sld_x.value:sld_x.value+110].reshape((110, 110)))
        plt.show()
sld_x.observe(update_C, 'value')
sld_y.observe(update_C, 'value')

update_C(None)
VBox([sld_x, sld_y, output])

In [ ]:
tSNE_2D(images_HM2, labels_pred)
# tSNE_3D(images_HM2, labels_pred)

## Perform clustering with SSC

In [ ]:
from supporting_files.ji_zhang import err_rate
from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import adjusted_rand_score as ari

predicted = labels_pred
print("Accuracy: ", str(1-err_rate(labels, predicted)))
print("NMI: ", str(nmi(labels, predicted, average_method="geometric")))
print("ARI: ", str(ari(labels, predicted)))

In [ ]:
import sys
from sklearn.cluster import SpectralClustering
# sc = SpectralClustering(n_clusters=20, random_state=0)
# labels_pred2 = sc.fit_predict(images_HM2, labels)
sc = SpectralClustering(n_clusters=20, random_state=0, affinity='precomputed')
labels_pred2 = sc.fit_predict(C_after, labels)